# basic/B5

## 几何

+ basic/B5 示例中的结构如下，包含多个不同的部分，比如 magnetic phys，drift chamber phys，hodoscope phys，em-calorimeter phys 和 hod-calorimeter phys。除了 magnetic phys 之外，其余均是 sensitive detector.
<img src="./fig/20240305_0.png" alt="" width="600" height="400"> 
+ 在构建几何时，用到了 G4PVPlacement()，G4PVReplica() 以及 G4PVParameterised()，并且均是针对同一逻辑体进行的拷贝，因此在可视化界面中也可以看到对应的 copy number，如下如所示。
<img src="./fig/20240305_1.png" alt="" width="220" height="420"> 

## 灵敏探测器

### 如何设置灵敏探测器 ？
+ 不同的灵敏探测器需要记录的信息不同，因此要有不同的灵敏探测器的类。
+ 灵敏探测器分为 hit，HitCollection，G4HCofThisEvent
  + hit 记录在 sensitive 探测器中每个 step 的信息，一个 sensitive 探测器将产生大量的 hit 对象；
  + HitCollection 记录 sensitive 探测器中产生的大量 hit 对象，是一个容器；
  + G4HCofThisEvent 是一个容器，多个 sensitive 探测器的 HitCollection 的对象保存在该容器中。
+ sensitive 探测器是如何工作的？
<img src="./fig/20240305_2.png" alt="" width="800" height="500"> 
+ sensitive 探测器需要实现其构造函数，Initialize()，ProcessHit() 以及 EndOfEvent() 函数。其中
  + 在构造函数中需要定义 HitCollection 对象的名字；
  + Initialize() 在每次 EventAction 开始前调用，在该函数中，将定义的 HitCollection 对象放到 G4HCofThisEvent 对象中；
  + ProcessHit() 和 EndOfEvent() 函数中将 hit 对象添加至 HitCollection 中。
+ 一般来说，我们需要的沉积在灵敏探测器中的总能量，因此通常都是在 ProcessHit() 和 EndOfEvent() 函数中将 hit 能量相加。正是由于一般只需要沉积的总能量，故而针对同一个灵敏探测器，在 HitCollection 中其实只需要保存一个 hit 即可，用于记录能量，位置，时间信息。因此可以考虑在 EndOfEvent() 函数中使用 insert 函数给 HitCollection 对象添加 hit 对象。
+ 此外，当同一个逻辑体被 copy 多了物理体，并且均是灵敏探测器时，这时一般在 Initialize() 函数中就根据 copy 的数目 insert 对应数目的 hit 对象，然后在 ProcessHit() 中再针对不同的 hit 对象处理。

### 在 EventAction 中如何使用灵敏探测器 ？
+ 首先需要知道灵敏探测器对应的 HitCollection 对象的编号，可以通过 `sd_manager->GetCollectionID("sd_name/hit_name")` 获得，其中 sd_name 对应灵敏探测器的名字，在 DetectorConstruction 中定义灵敏探测器时需设置，hit_name 对应 hit 的名称，在对应的 Hit 类中设置。在得知 HitCollection 对象的编号后，用 `event->GetHCofThisEvent()->GetHC(id)` 得到具体的 HitCollection 对象，由此再得到具体的 Hit 对象，继而可以提取相应的物理信息。